In [4]:
import requests
import re
import csv
import pandas
import time
import numpy

In [2]:
def has_test_case(line):
   assert isinstance(line, str) ##断言，确保line是一个字符串
   pattern = "^diff --git.*Test.*$"  ##定义正则表达式
   ## ^ 匹配行的开头， 。* 匹配任意字符 Test 匹配Test，再匹配任意数量的字符，刀乐符匹配结尾
   # re.match调用re的接口返回匹配对象，，bool将返回结果变为布尔值
   return bool(re.match(pattern, line))

In [3]:
def is_func(str):
    ##传入一个字符串，然后判断这一行里面有没有属于下面关键字的字串，如果有返回位置，没有返回零
    # 方法的修饰符用来判断一个方法
    funcs = ['public','protected','private','static','final','synchronized','native','abstract']
    for s in funcs:
        index = str.find(s)
        if (index != -1):
            return index
    return -1

In [5]:
def get_repo_contents(prj, path=""):
    ## http请求获得仓库的的内容
    url = f"https://api.github.com/repos/prj/contents/{path}"
    time.sleep(6)
    print(123)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    ##以json串的形式返回
    else:
        print(f"Failed to retrieve repository contents for {path}.")
        return None

In [6]:
def get_test_files(prj, path="", goal = ""):
    #return False
    def match_test_file(filename):
        assert isinstance(filename, str)
        pattern = "^Test" + re.escape(str) + "$" ##构造正则匹配，匹配test文件
        return bool(re.match(pattern, filename))
        # return True

    contents = get_repo_contents(prj, path)
    # file = open(output_file, "a")
    if contents is not None:
        for item in contents:
            if item['type'] == 'file' and match_test_file(item['name']):
                return True
            elif item['type'] == 'dir':
                return get_test_files(prj, item['path']) ##这里也是一个布尔值
                # file.write(item['name'])
    # file.close()
    return False

In [39]:
# 示例url
url='https://github.com/alibaba/fastjson/commit/f5903fa56497c00ed0703ac875b511f9bd5f1d8e'
res=requests.get(url)
time.sleep(6)
res

<Response [200]>

In [40]:
# 示例url
url='https://github.com/alibaba/fastjson/commit/f5903fa56497c00ed0703ac875b511f9bd5f1d8e.diff'
res=requests.get(url)
time.sleep(6)
res

<Response [200]>

In [41]:
s = res.text.splitlines(keepends = False)
s

['diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 'index 9b8c0e3c17..77f2d63d98 100755',
 '--- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 '+++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 '@@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {',
 '             componentType = componentClass = clazz.getComponentType();',
 '         }',
 '         JSONArray array = new JSONArray();',
 '-        parser.parseArray(componentClass, array, fieldName);',
 '+        parser.parseArray(componentType, array, fieldName);',
 ' ',
 '         return (T) toObjectArray(parser, componentClass, array);',
 '     }',
 'diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java',
 'index 2ceb034b87..aec293d74d 100644',
 '--- a/src/test/jav

In [42]:
    file = 0
    file_java = 0
    func = 0
    hunk = 0
    ischange = 0
    isTestcase = 0
    iscomment = 0
        # 使用set去重
    funcset = set()

In [43]:
for st in s:
    print(st)
    print("[+]")

diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[+]
index 9b8c0e3c17..77f2d63d98 100755
[+]
--- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[+]
+++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[+]
@@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {
[+]
             componentType = componentClass = clazz.getComponentType();
[+]
         }
[+]
         JSONArray array = new JSONArray();
[+]
-        parser.parseArray(componentClass, array, fieldName);
[+]
+        parser.parseArray(componentType, array, fieldName);
[+]
 
[+]
         return (T) toObjectArray(parser, componentClass, array);
[+]
     }
[+]
diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
[+]
index 2ceb034b87..aec293d74d 100644
[+]
--- a/src/test/java/

In [ ]:
# 第一个元素是一个字符串，其中包含了一段Git格式的diff信息，用于表示文件的变更情况。让我解释一下这段diff信息：
# 
# 'diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java'：这是Git格式的diff头部信息，表示文件变更前后的路径。在这个例子中，它表示一个Java文件的路径从 a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java 变更为 b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java。
# 'index 9b8c0e3c17..77f2d63d98 100755'：这行信息通常出现在diff头部，它指示了变更前后文件的版本号或哈希值。
# '--- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java' 和 '+++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java'：这两行信息指示了变更前后文件的路径。
# '@@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {'：这是diff的行号信息，表示变更前后的行号范围。
# ' componentType = componentClass = clazz.getComponentType();' 和 ' parser.parseArray(componentType, array, fieldName);'：这两行是具体的代码变更，表示文件中的某些行在变更前后发生了改变。

In [ ]:
# Git格式的diff通常用于表示文件的变更情况，比如在版本控制系统中提交的代码变更。它具有一定的框架和格式，可以通过阅读和理解来了解文件的变更情况。下面是Git格式diff的一般框架和阅读方式：
# 
# 框架：
# 头部信息：通常以 diff --git 开始，包含了变更前后文件的路径和版本号信息。
# 变更信息：列出了文件的变更情况，包括增加、删除和修改。
# 行号信息：以 @@ 开始，指示了变更前后文件的行号范围。
# 具体的代码变更：列出了文件中发生变更的具体行，包括变更前后的代码内容。
# 阅读方式：
# 头部信息：
# 查看文件路径和版本号信息，了解变更前后文件的位置和版本。
# 变更信息：
# 查看每个文件的变更情况，包括新增、删除和修改。
# 对于新增的文件，可以查看新增的文件路径和内容。
# 对于删除的文件，可以查看删除的文件路径和内容。
# 对于修改的文件，可以查看变更前后的文件路径和内容。
# 行号信息：
# 查看行号信息，了解变更前后的行号范围。
# 具体的代码变更：
# 根据行号信息查看具体的代码变更，包括变更前后的代码内容。
# 示例：
# diff
# Copy code
# diff --git a/src/main/java/com/example/MyClass.java b/src/main/java/com/example/MyClass.java
# index 123456..789abc 100644
# --- a/src/main/java/com/example/MyClass.java
# +++ b/src/main/java/com/example/MyClass.java
# @@ -1,5 +1,5 @@
#  public class MyClass {
# -    private String name;
# +    private String firstName;
#  
# -    public MyClass(String name) {
# +    public MyClass(String firstName) {
#          this.name = name;
#      }
#  }
# 在这个示例中：
# 
# 头部信息指示了变更前后文件的路径和版本号。
# 变更信息表示文件发生了修改，从 -1,5 +1,5 可以看出是从第1行开始，修改了5行。
# 行号信息 @@ -1,5 +1,5 @@ 指示了变更前后的行号范围。
# 具体的代码变更显示了修改前后的代码内容，比如将 private String name; 修改为 private String firstName;。

In [44]:
    for st in s:
        # 删除前导空格，方便后面的操作
        st = re.sub(r' {2,}', '', st)
        print(st)
        #缩进被干掉了

diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
index 9b8c0e3c17..77f2d63d98 100755
--- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
+++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
@@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {
componentType = componentClass = clazz.getComponentType();
}
JSONArray array = new JSONArray();
-parser.parseArray(componentClass, array, fieldName);
+parser.parseArray(componentType, array, fieldName);
 
return (T) toObjectArray(parser, componentClass, array);
}
diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
index 2ceb034b87..aec293d74d 100644
--- a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
+++ b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
@@ -10,7 +10,18 @@
*/

In [45]:
    for st in s:
        # 删除前导空格，方便后面的操作
        st = re.sub(r' {2,}', '', st)
        #html代码的每一行的所有空格全部替换

        # file
        if (st.startswith("diff") == True):
            print("[+]True:",st)
            

[+]True: diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[+]True: diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java


In [46]:
new_s1=[]
for st in s:
    st = re.sub(r' {2,}', '', st)
    if st.startswith("diff") == True:
        new_s1.append(st)
new_s1

['diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 'diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java']

In [56]:
for st in new_s1:
    match = re.search(r'/([^/]*)$',st)#匹配最后一个斜杠后面的内容
    if match:
        filename = match.group(1) #拿到filename
        print("[filename]:",filename)
    match = re.search(r'/([^/]*)$', url)  #从url提取最后一个斜杠的内容，我的感觉是哈希？
    if match:
        html_url = match.group(1)
        print('[html_url]:',html_url)

[filename]: ObjectArrayCodec.java
[html_url]: f5903fa56497c00ed0703ac875b511f9bd5f1d8e.diff
[filename]: Issue1005.java
[html_url]: f5903fa56497c00ed0703ac875b511f9bd5f1d8e.diff


In [48]:
s

['diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 'index 9b8c0e3c17..77f2d63d98 100755',
 '--- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 '+++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java',
 '@@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {',
 '             componentType = componentClass = clazz.getComponentType();',
 '         }',
 '         JSONArray array = new JSONArray();',
 '-        parser.parseArray(componentClass, array, fieldName);',
 '+        parser.parseArray(componentType, array, fieldName);',
 ' ',
 '         return (T) toObjectArray(parser, componentClass, array);',
 '     }',
 'diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java',
 'index 2ceb034b87..aec293d74d 100644',
 '--- a/src/test/jav

In [54]:
for st in s:
    
    if has_test_case(st):
        print("[test_case]:",st)

In [53]:
for st in s:
    if st.endswith(".java"):
        print("[java]:",st)

[java]: diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[java]: --- a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[java]: +++ b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[java]: diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
[java]: --- a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
[java]: +++ b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java


In [57]:
filename='ObjectArrayCodec.java'
html_url= 'f5903fa56497c00ed0703ac875b511f9bd5f1d8e.diff'

    

In [58]:
    def match_test_file(filename):
        assert isinstance(filename, str)
        pattern = "^Test" + re.escape(str) + "$" ##构造正则匹配，匹配test文件
        return bool(re.match(pattern, filename))
        # return True

In [59]:
contents = get_repo_contents(html_url)
contents
#这里调用没成功，上面是github的接口吗？这里应该返回整个仓库

123
Failed to retrieve repository contents for .


In [61]:
for st in s:
      if st.startswith('-') == False and is_func(st) != -1 and st.find('(') != -1 and st.find(')') != -1 :
          print("[st]:",st)
#找到函数
      if st.startswith('-') == False and is_func(st) != -1 and st.find('(') != -1 and st.find(')') != -1 :
          index1 = is_func(st)
          index2 = st.find(')')
          stg = st[index1 : index2 + 1] 
          print("[stg]:",stg)

[st]: @@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {
[stg]: public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName)
[st]:      public void test_for_issue() throws Exception {
[stg]: public void test_for_issue()
[st]: +    public void test_for_List() throws Exception {
[stg]: public void test_for_List()
[st]: @@ -19,6 +30,10 @@ public void test_for_issue() throws Exception {
[stg]: public void test_for_issue()


In [62]:
for st in s:
     if (st.startswith('+') == 0) and (st.startswith('-') == 0):
         print("[st]:",st)
##非修改

[st]: diff --git a/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java b/src/main/java/com/alibaba/fastjson/serializer/ObjectArrayCodec.java
[st]: index 9b8c0e3c17..77f2d63d98 100755
[st]: @@ -174,7 +174,7 @@ public <T> T deserialze(DefaultJSONParser parser, Type type, Object fieldName) {
[st]:              componentType = componentClass = clazz.getComponentType();
[st]:          }
[st]:          JSONArray array = new JSONArray();
[st]:  
[st]:          return (T) toObjectArray(parser, componentClass, array);
[st]:      }
[st]: diff --git a/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java b/src/test/java/com/alibaba/json/bvt/bug/Issue1005.java
[st]: index 2ceb034b87..aec293d74d 100644
[st]: @@ -10,7 +10,18 @@
[st]:   */
[st]:  public class Issue1005 extends TestCase {
[st]:      public void test_for_issue() throws Exception {
[st]:          assertNotNull(model.values);
[st]:          assertEquals(3, model.values.size());
[st]:          assertEquals(Byte.class, mode

In [64]:
new_s2=[]
for st in s:
    if  ((st.startswith("+") == True and st.startswith("+++") == False) 
            or (st.startswith("-") == True and st.startswith("---") == False)) and (ischange == 0):
        print("[change]:",st)
        new_s2.append(st)

[change]: -        parser.parseArray(componentClass, array, fieldName);
[change]: +        parser.parseArray(componentType, array, fieldName);
[change]: -        Model model = JSON.parseObject("{\"values\":[1,2,3]}", Model.class);
[change]: +        Model model = JSON.parseObject("{\"values\":[[1,2,3]]}", Model.class);
[change]: +
[change]: +        assertNotNull(model.values);
[change]: +        assertEquals(3, model.values[0].size());
[change]: +        assertEquals(Byte.class, model.values[0].get(0).getClass());
[change]: +        assertEquals(Byte.class, model.values[0].get(1).getClass());
[change]: +        assertEquals(Byte.class, model.values[0].get(2).getClass());
[change]: +    }
[change]: +
[change]: +    public void test_for_List() throws Exception {
[change]: +        Model2 model = JSON.parseObject("{\"values\":[1,2,3]}", Model2.class);
[change]: +
[change]: +        public List<Byte>[] values;
[change]: +    }
[change]: +
[change]: +    public static class Model2 {


In [65]:
for st in new_s2:
     if (st.find("import") != -1) :
         print("[st]:",st)

In [66]:
for st in new_s2:
    if st.find("//") == 1:
         print("[st]:",st)

In [67]:
for st in new_s2:
    if (st.find('/**') == 1) or (st.find('/*') == 1):
        print("[st]:",st)
        

In [68]:
for st in new_s2:
    if len(st)==1:
        print("[st]:",st)

[st]: +
[st]: +
[st]: +
[st]: +
